# 気象データべース地上観測の日別または時別複数年連続ファイルを各年別に分割
### Splitting a GWO Houly or Daily data file containing multiple-year into a set of each-year file
**Author: Jun Sasaki, Coded on December 14, 2017, Revised on Sep. 17, 2018** <br>
気象データベース地上観測（GWO）DVD時別値は，1961年から1990年までは3時間間隔データとなっている．GWOのDVDから1地点1ファイルとして複数年一括出力したものを年毎に分ける．1991年以降は1時間間隔となっている．本コードはどちらにも，あるいは混在していても適用可能である．<br>
このほかに日別値にも同様に対応した．<br>
【解説】GWOのDVDでは1測点について，連続した年号を一括で出力可能．このように1ファイルとして出力したものを年毎のファイルに分割するのが目的
#### 注意
* 入力csvファイルのエンコードはSHIFT-JIS，CRLF，分割後の年別ファイルのエンコードはUTF-8，LFである．
* nkfを用い，linefeedをLinux形式としている.
* データベースはSQLViewer7を立ち上げ，「有効データ」および「閾値ソート」のチェックを外し，「全データベース項目」にチェックを入れ，観測所を一つ選び，全期間を指定して実行する．
* csvファイルに保存する際は，抽出DB画面の「編集」の「列ラベル出力」にチェックを入れてから，保存する．
* Windows版では出力が自動的にShift-JISになったため，pandas.to_csvでエンコードをUTF-8に指定する必要がある．
* 時別値は1日の最後のデータ時刻が24時のため，年末24時のデータは翌年年初0時のデータを意味する．

In [1]:
import pandas as pd
import subprocess
import os
import sys
import glob

In [2]:
stn_dict={"稚内":"Wakkanai", "北見枝幸":"Kitamiesashi", "羽幌":"Haboro", "雄武":"Oumu", "留萌":"Rumoi", "旭川":"Asahikawa", \
          "網走":"Abashiri", "小樽":"Otaru", "札幌":"Sapporo", "岩見沢":"Iwamizawa", "帯広":"Obihiro", "釧路":"Kushiro", \
          "根室":"Nemuro", "寿都":"Suttu", "室蘭":"Muroran", "苫小牧":"Tomakomai", \
          "浦河":"Urakawa", "江差":"Esashi", "函館":"Hakodate", "倶知安":"Kutchan", "紋別":"Monbetsu", "広尾":"Hiroo", \
          "大船渡":"Ofunato", "新庄":"Shinjo", "若松":"Wakamatsu", "深浦":"Fukaura", \
          "青森":"Aomori", "むつ":"Mutsu", "八戸":"Hachinohe", "秋田":"Akita", "盛岡":"Morioka", "宮古":"Miyako", \
          "酒田":"Sakata", "山形":"Yamagata", "仙台":"Sendai", "石巻":"Ishinomaki", \
          "福島":"Fukushima", "白河":"Shirakawa", "小名浜":"Onahama", "輪島":"Wajima", "相川":"Aikawa", "新潟":"Niigata", \
          "金沢":"Kanazawa", "伏木":"Fushiki", "富山":"Toyama", "長野":"Nagano", \
          "高田":"Takada", "宇都宮":"Utsunomiya", "福井":"Fukui", "高山":"Takayama", "松本":"Matsumoto", \
          "諏訪":"Suwa", "軽井沢":"Karuizawa", "前橋":"Maebashi", "熊谷":"Kumagaya", "水戸":"Mito", \
          "敦賀":"Tsuruga", "岐阜":"Gifu", "名古屋":"Nagoya", "飯田":"Iida", "甲府":"Kofu", \
          "河口湖":"Kawaguchiko", "秩父":"Chichibu", "館野":"Tateno", "銚子":"Choshi", "上野":"Ueno", \
          "津":"Tsu", "伊良湖":"Irago", "浜松":"Hamamatsu", "御前崎":"Omaezaki", "静岡":"Shizuoka", \
          "三島":"Mishima", "東京":"Tokyo", "尾鷲":"Owase", "石廊崎":"Irozaki", "網代":"Ajiro", \
          "横浜":"Yokohama", "館山":"Tateyama", "勝浦":"Katsuura", "大島":"Oshima", "三宅島":"Miyakejima", \
          "八丈島":"Hachijojima", "千葉":"Chiba", "四日市":"Yokkaichi", "日光":"Nikko", "西郷":"Saigo", \
          "松江":"Matsue", "境":"Sakai", "米子":"Yonago", "鳥取":"Tottori", "豊岡":"Toyooka", "舞鶴":"Maiduru", \
          "伊吹山":"Ibukiyama", "萩":"Hagi", "浜田":"Hamada", "津山":"Tsuyama", \
          "京都":"Kyoto", "彦根":"Hikone", "下関":"Shimonoseki", "広島":"Hiroshima", "呉":"Kure", \
          "福山":"Fukuyama", "岡山":"Okayama", "姫路":"Himeji", "神戸":"Kobe", "大阪":"Osaka", \
          "洲本":"Sumoto", "和歌山":"Wakayama", "潮岬":"Shionomisaki", "奈良":"Nara", "山口":"Yamaguchi", \
          "厳原":"Izuhara", "平戸":"Hirado", "福岡":"Fukuoka", "飯塚":"Iiduka", "佐世保":"Sasebo", \
          "佐賀":"Saga", "日田":"Hita", "大分":"Oita", "長崎":"Nagasaki", "熊本":"Kumamoto", \
          "阿蘇山":"Asosan", "延岡":"Nobeoka", "阿久根":"Akune", "人吉":"Hitoyoshi", "鹿児島":"Kagoshima", \
          "都城":"Miyakonojo", "宮崎":"Miyazaki", "枕崎":"Makurazaki", "油津":"Aburatsu", "屋久島":"Yakushima", \
          "種子島":"Tanegashima", "牛深":"Ushibuka", "福江":"Fukue", "松山":"Matsuyama", "多度津":"Tadotsu", \
          "高松":"Takamatsu", "宇和島":"Uwajima", "高知":"Kochi", "剣山":"Tsurugisan", "徳島":"Tokushima", \
          "宿毛":"Sukumo", "清水":"Shimizu", "室戸岬":"Murotomisaki", "名瀬":"Nase", "与那国島":"Yonakunijima", \
          "石垣島":"Ishigakijima", "宮古島":"Miyakojima", "久米島":"Kumejima", "那覇":"Naha", "名護":"Nago", \
          "沖永良部":"Okinoerabu", "南大東島":"Minamidaitojima", "父島":"Chichijima", "南鳥島":"Minamitorishima"}

In [7]:
def GWO_div_year(stn="Tokyo", year_ini=None, year_end=None, dir="../GWO/"):
    '''Divide a file containing continuous years data into each year file'''
    ### File path for reading file, e.g., Tokyo1961-1990.csv (UTF-8)
    dirpath=dir + stn + "/"
    if not os.path.isdir(dirpath):
        print('Error: No such a directory')
        sys.exit()
    fpath = glob.glob(dirpath + '*-*.csv')
    if not year_ini == None and not year_end == None:  ### year_iniとyear_endが与えられている場合
        fpath=dir + stn +"/" + stn + str(year_ini) + "-" + str(year_end) + ".csv"
        print(fpath)
    else: ### year_iniとyear_endの少なくとも一方がNoneの場合
        ### 時別値は2つのファイルが存在
        if len(fpath) == 1: ### 日別値の場合
            fpath=fpath[0]
        elif len(fpath) > 1:
            if year_end == 1990:
                fpath = glob.glob(dirpath + '*-1990*.csv')[0]
            elif year_ini == 1991:
                fpath = glob.glob(dirpath + '*1991-*.csv')[0]
            else:
                print("Error: Number of all-year csv files cannot be selected.")
                print(fpath)
                sys.exit()
        else:
            print("Error: Number of all-year csv files cannot be selected.")
            print(fpath)
            sys.exit()
    print("Reading ", fpath)
    df=pd.read_csv(fpath, header=None, dtype="str", encoding="SHIFT-JIS")
    if year_ini == None:
        year_ini = int(df.iloc[0,3])
    if year_end == None:
        year_end = int(df.iloc[-1,3])
    for year in range(year_ini, year_end + 1):
        df_year = df[df.iloc[:,3] == str(year)]  ### Column 3 of df is year.
        fpath_year = dir + stn + "/" + stn + str(year) + ".csv"  ### output CSV file path for each year
        print("Creating", fpath_year)
        df_year.to_csv(fpath_year, header=None, index=False, encoding="utf-8")  ### エンコードをutf-8とする
        ### Converting to linefeed of LF  ### Linux互換とするため，改行コードをLFにする
        cmd = 'nkf -w -Lu --overwrite ' + fpath_year
        print("Converting to LF", cmd)
        subprocess.call(cmd)

## 時別値ファイルを分割する
### ../GWO/Hourly/以下に地点名のディレクトリに，Tokyo1961-1990.csvといったデータファイルが要存在

In [ ]:
stn="Tokyo"  ### "Tokyo" or "Yokohama" or "Chiba" or "Tateyama"
year_ini=1961  ### データの存在する最初の年は地点により異なるので，要設定
GWO_div_year(stn=stn, year_ini=None, year_end=1990)
GWO_div_year(stn=stn, year_ini=1991, year_end=None)

### すべての観測点におけるすべての観測年の時別値ファイルに一気に分割する

In [12]:
stns = list(stn_dict.values())
dir="../GWO/Hourly/"
for stn in stns:
    #GWO_div_year(stn=stn, year_ini=None, year_end=1990, dir=dir)
    GWO_div_year(stn=stn, year_ini=1991, year_end=None, dir=dir)

Reading  ../Hourly/Wakkanai\Wakkanai1991-2017.csv
Creating ../Hourly/Wakkanai/Wakkanai1991.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1991.csv
Creating ../Hourly/Wakkanai/Wakkanai1992.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1992.csv
Creating ../Hourly/Wakkanai/Wakkanai1993.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1993.csv
Creating ../Hourly/Wakkanai/Wakkanai1994.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1994.csv
Creating ../Hourly/Wakkanai/Wakkanai1995.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1995.csv
Creating ../Hourly/Wakkanai/Wakkanai1996.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1996.csv
Creating ../Hourly/Wakkanai/Wakkanai1997.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai/Wakkanai1997.csv
Creating ../Hourly/Wakkanai/Wakkanai1998.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Wakkanai

Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2001.csv
Creating ../Hourly/Haboro/Haboro2002.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2002.csv
Creating ../Hourly/Haboro/Haboro2003.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2003.csv
Creating ../Hourly/Haboro/Haboro2004.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2004.csv
Creating ../Hourly/Haboro/Haboro2005.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2005.csv
Creating ../Hourly/Haboro/Haboro2006.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2006.csv
Creating ../Hourly/Haboro/Haboro2007.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2007.csv
Creating ../Hourly/Haboro/Haboro2008.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2008.csv
Creating ../Hourly/Haboro/Haboro2009.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Haboro/Haboro2009.csv
Creating ../Hourly/Habor

Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa1995.csv
Creating ../Hourly/Asahikawa/Asahikawa1996.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa1996.csv
Creating ../Hourly/Asahikawa/Asahikawa1997.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa1997.csv
Creating ../Hourly/Asahikawa/Asahikawa1998.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa1998.csv
Creating ../Hourly/Asahikawa/Asahikawa1999.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa1999.csv
Creating ../Hourly/Asahikawa/Asahikawa2000.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa2000.csv
Creating ../Hourly/Asahikawa/Asahikawa2001.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa2001.csv
Creating ../Hourly/Asahikawa/Asahikawa2002.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Asahikawa/Asahikawa2002.csv
Creating ../Hourly/Asahikawa/Asahikawa2003.csv


Creating ../Hourly/Otaru/Otaru2011.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2011.csv
Creating ../Hourly/Otaru/Otaru2012.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2012.csv
Creating ../Hourly/Otaru/Otaru2013.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2013.csv
Creating ../Hourly/Otaru/Otaru2014.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2014.csv
Creating ../Hourly/Otaru/Otaru2015.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2015.csv
Creating ../Hourly/Otaru/Otaru2016.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2016.csv
Creating ../Hourly/Otaru/Otaru2017.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Otaru/Otaru2017.csv
Reading  ../Hourly/Sapporo\Sapporo1991-2017.csv
Creating ../Hourly/Sapporo/Sapporo1991.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Sapporo/Sapporo1991.csv
Creating ../Hourly/Sapporo/Sapporo1992.csv
Converting to LF nkf -w -Lu -

Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro1998.csv
Creating ../Hourly/Obihiro/Obihiro1999.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro1999.csv
Creating ../Hourly/Obihiro/Obihiro2000.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2000.csv
Creating ../Hourly/Obihiro/Obihiro2001.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2001.csv
Creating ../Hourly/Obihiro/Obihiro2002.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2002.csv
Creating ../Hourly/Obihiro/Obihiro2003.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2003.csv
Creating ../Hourly/Obihiro/Obihiro2004.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2004.csv
Creating ../Hourly/Obihiro/Obihiro2005.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihiro2005.csv
Creating ../Hourly/Obihiro/Obihiro2006.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Obihiro/Obihir

...

Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2001.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2002.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2002.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2003.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2003.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2004.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2004.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2005.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2005.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2006.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2006.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2007.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Murotomisaki/Murotomisaki2007.csv
Creating ../Hourly/Murotomisaki/Murotomisaki2008.csv
Converting to LF nkf -w -Lu --overwrite .

Creating ../Hourly/Yonakunijima/Yonakunijima2013.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Yonakunijima/Yonakunijima2013.csv
Creating ../Hourly/Yonakunijima/Yonakunijima2014.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Yonakunijima/Yonakunijima2014.csv
Creating ../Hourly/Yonakunijima/Yonakunijima2015.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Yonakunijima/Yonakunijima2015.csv
Creating ../Hourly/Yonakunijima/Yonakunijima2016.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Yonakunijima/Yonakunijima2016.csv
Creating ../Hourly/Yonakunijima/Yonakunijima2017.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Yonakunijima/Yonakunijima2017.csv
Reading  ../Hourly/Ishigakijima\Ishigakijima1991-2017.csv
Creating ../Hourly/Ishigakijima/Ishigakijima1991.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Ishigakijima/Ishigakijima1991.csv
Creating ../Hourly/Ishigakijima/Ishigakijima1992.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Ishigakijima/Ishi

Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1992.csv
Creating ../Hourly/Kumejima/Kumejima1993.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1993.csv
Creating ../Hourly/Kumejima/Kumejima1994.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1994.csv
Creating ../Hourly/Kumejima/Kumejima1995.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1995.csv
Creating ../Hourly/Kumejima/Kumejima1996.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1996.csv
Creating ../Hourly/Kumejima/Kumejima1997.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1997.csv
Creating ../Hourly/Kumejima/Kumejima1998.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1998.csv
Creating ../Hourly/Kumejima/Kumejima1999.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Kumejima/Kumejima1999.csv
Creating ../Hourly/Kumejima/Kumejima2000.csv
Converting to LF nkf -w -Lu --ov

Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2012.csv
Creating ../Hourly/Nago/Nago2013.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2013.csv
Creating ../Hourly/Nago/Nago2014.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2014.csv
Creating ../Hourly/Nago/Nago2015.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2015.csv
Creating ../Hourly/Nago/Nago2016.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2016.csv
Creating ../Hourly/Nago/Nago2017.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Nago/Nago2017.csv
Reading  ../Hourly/Okinoerabu\Okinoerabu1991-2017.csv
Creating ../Hourly/Okinoerabu/Okinoerabu1991.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Okinoerabu/Okinoerabu1991.csv
Creating ../Hourly/Okinoerabu/Okinoerabu1992.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Okinoerabu/Okinoerabu1992.csv
Creating ../Hourly/Okinoerabu/Okinoerabu1993.csv
Converting to LF nkf -w -Lu --overwrite ../Hou

Creating ../Hourly/Chichijima/Chichijima1991.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1991.csv
Creating ../Hourly/Chichijima/Chichijima1992.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1992.csv
Creating ../Hourly/Chichijima/Chichijima1993.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1993.csv
Creating ../Hourly/Chichijima/Chichijima1994.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1994.csv
Creating ../Hourly/Chichijima/Chichijima1995.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1995.csv
Creating ../Hourly/Chichijima/Chichijima1996.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1996.csv
Creating ../Hourly/Chichijima/Chichijima1997.csv
Converting to LF nkf -w -Lu --overwrite ../Hourly/Chichijima/Chichijima1997.csv
Creating ../Hourly/Chichijima/Chichijima1998.csv
Converting to LF nkf -w -Lu --overwrite ../Hourl

## 日別値ファイルを分割する
時別値と同様に複数年のcsvファイルをデフォルトのSHIFT-JIS，CRLFで準備する．例：Tokyo1961-2017.csv

In [ ]:
stn="Matsue"
year_ini=None
year_end=None
dir="../GWO/Daily/"
GWO_div_year(stn=stn, year_ini=year_ini, year_end=year_end, dir=dir)

### すべての観測点におけるすべての観測年の日別値ファイルに一気に分割する

In [10]:
stns = list(stn_dict.values())
dir="../GWO/Daily/"
for stn in stns:
    GWO_div_year(stn=stn, year_ini=None, year_end=None, dir=dir)

Reading  ../Daily/Wakkanai\Wakkanai1961-2017.csv
Creating ../Daily/Wakkanai/Wakkanai1961.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1961.csv
Creating ../Daily/Wakkanai/Wakkanai1962.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1962.csv
Creating ../Daily/Wakkanai/Wakkanai1963.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1963.csv
Creating ../Daily/Wakkanai/Wakkanai1964.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1964.csv
Creating ../Daily/Wakkanai/Wakkanai1965.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1965.csv
Creating ../Daily/Wakkanai/Wakkanai1966.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1966.csv
Creating ../Daily/Wakkanai/Wakkanai1967.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1967.csv
Creating ../Daily/Wakkanai/Wakkanai1968.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Wakkanai/Wakkanai1968.csv

Creating ../Daily/Kitamiesashi/Kitamiesashi1971.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1971.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1972.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1972.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1973.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1973.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1974.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1974.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1975.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1975.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1976.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1976.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1977.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Kitamiesashi/Kitamiesashi1977.csv
Creating ../Daily/Kitamiesashi/Kitamiesashi1978.csv
Con

Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1978.csv
Creating ../Daily/Haboro/Haboro1979.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1979.csv
Creating ../Daily/Haboro/Haboro1980.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1980.csv
Creating ../Daily/Haboro/Haboro1981.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1981.csv
Creating ../Daily/Haboro/Haboro1982.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1982.csv
Creating ../Daily/Haboro/Haboro1983.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1983.csv
Creating ../Daily/Haboro/Haboro1984.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1984.csv
Creating ../Daily/Haboro/Haboro1985.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1985.csv
Creating ../Daily/Haboro/Haboro1986.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Haboro/Haboro1986.csv
Creating ../Daily/Haboro/Haboro1987.csv
C

Creating ../Daily/Oumu/Oumu2000.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2000.csv
Creating ../Daily/Oumu/Oumu2001.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2001.csv
Creating ../Daily/Oumu/Oumu2002.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2002.csv
Creating ../Daily/Oumu/Oumu2003.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2003.csv
Creating ../Daily/Oumu/Oumu2004.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2004.csv
Creating ../Daily/Oumu/Oumu2005.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2005.csv
Creating ../Daily/Oumu/Oumu2006.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2006.csv
Creating ../Daily/Oumu/Oumu2007.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2007.csv
Creating ../Daily/Oumu/Oumu2008.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Oumu/Oumu2008.csv
Creating ../Daily/Oumu/Oumu2009.csv
Converting to LF nkf -w -Lu --overwri

Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1965.csv
Creating ../Daily/Asahikawa/Asahikawa1966.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1966.csv
Creating ../Daily/Asahikawa/Asahikawa1967.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1967.csv
Creating ../Daily/Asahikawa/Asahikawa1968.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1968.csv
Creating ../Daily/Asahikawa/Asahikawa1969.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1969.csv
Creating ../Daily/Asahikawa/Asahikawa1970.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1970.csv
Creating ../Daily/Asahikawa/Asahikawa1971.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1971.csv
Creating ../Daily/Asahikawa/Asahikawa1972.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Asahikawa/Asahikawa1972.csv
Creating ../Daily/Asahikawa/Asahikawa1973.csv
Converting to LF

...

Creating ../Daily/Naha/Naha1965.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1965.csv
Creating ../Daily/Naha/Naha1966.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1966.csv
Creating ../Daily/Naha/Naha1967.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1967.csv
Creating ../Daily/Naha/Naha1968.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1968.csv
Creating ../Daily/Naha/Naha1969.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1969.csv
Creating ../Daily/Naha/Naha1970.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1970.csv
Creating ../Daily/Naha/Naha1971.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1971.csv
Creating ../Daily/Naha/Naha1972.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1972.csv
Creating ../Daily/Naha/Naha1973.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Naha/Naha1973.csv
Creating ../Daily/Naha/Naha1974.csv
Converting to LF nkf -w -Lu --overwri

Creating ../Daily/Nago/Nago2000.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2000.csv
Creating ../Daily/Nago/Nago2001.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2001.csv
Creating ../Daily/Nago/Nago2002.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2002.csv
Creating ../Daily/Nago/Nago2003.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2003.csv
Creating ../Daily/Nago/Nago2004.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2004.csv
Creating ../Daily/Nago/Nago2005.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2005.csv
Creating ../Daily/Nago/Nago2006.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2006.csv
Creating ../Daily/Nago/Nago2007.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2007.csv
Creating ../Daily/Nago/Nago2008.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Nago/Nago2008.csv
Creating ../Daily/Nago/Nago2009.csv
Converting to LF nkf -w -Lu --overwri

Creating ../Daily/Minamidaitojima/Minamidaitojima1961.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1961.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1962.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1962.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1963.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1963.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1964.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1964.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1965.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1965.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1966.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaitojima/Minamidaitojima1966.csv
Creating ../Daily/Minamidaitojima/Minamidaitojima1967.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamidaito

Creating ../Daily/Chichijima/Chichijima1969.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1969.csv
Creating ../Daily/Chichijima/Chichijima1970.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1970.csv
Creating ../Daily/Chichijima/Chichijima1971.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1971.csv
Creating ../Daily/Chichijima/Chichijima1972.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1972.csv
Creating ../Daily/Chichijima/Chichijima1973.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1973.csv
Creating ../Daily/Chichijima/Chichijima1974.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1974.csv
Creating ../Daily/Chichijima/Chichijima1975.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chichijima1975.csv
Creating ../Daily/Chichijima/Chichijima1976.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Chichijima/Chi

Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1985.csv
Creating ../Daily/Minamitorishima/Minamitorishima1986.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1986.csv
Creating ../Daily/Minamitorishima/Minamitorishima1987.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1987.csv
Creating ../Daily/Minamitorishima/Minamitorishima1988.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1988.csv
Creating ../Daily/Minamitorishima/Minamitorishima1989.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1989.csv
Creating ../Daily/Minamitorishima/Minamitorishima1990.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1990.csv
Creating ../Daily/Minamitorishima/Minamitorishima1991.csv
Converting to LF nkf -w -Lu --overwrite ../Daily/Minamitorishima/Minamitorishima1991.csv
Creating ../Daily/Minamitoris